In [ ]:
JS | Header (End)
_________________

<!-- Finsweet - Load more -->
<script async src="https://cdn.jsdelivr.net/npm/@finsweet/attributes-cmsload@1/cmsload.js"></script>

In [ ]:
JS | Footer (Before)
____________________

<!-- CMS collection lists -->
<!-- Apply attributes to individual CMS items -->
<script>
  window.addEventListener("load", function () {  // Wait until all scripts/resources are loaded
    function applyAttributesToItems(selector, attributeName) {
      var items = document.querySelectorAll(selector); // Select all items
      if (items.length === 0) {
        // console.warn('No items found for selector:', selector);
      }
      items.forEach(function (item) {
        var attributeField = item.getAttribute('cms-item'); // Use the cms-item attribute
        if (attributeField) { // Check if the cms-item attribute exists
          item.setAttribute(attributeName, attributeField); // Set the custom attribute
          // console.log(`Set ${attributeName} to ${attributeField} for item`, item);
        } else {
          // console.warn('cms-item attribute not found in item:', item); // Warn if cms-item is not found
        }
      });
    }

    // Apply attributes to project and process items
    applyAttributesToItems('.project--list-item', 'project--attribute');
    applyAttributesToItems('.process--list-item', 'process--attribute');
  });
</script>

<!-- Hero -->
<!-- GSAP Marquee -->
<script>
  window.addEventListener("load", function () {  // Wait until all scripts/resources are loaded
    const marqueeTrack = document.querySelector('.marquee--track'); // Select marquee track
    const items = Array.from(marqueeTrack.children); // Get all child items

    // Durations for desktop and mobile (in seconds)
    const desktopDuration = 40, mobileDuration = 45;

    // Device detection, including tablets
    const isMobile = /Android|webOS|iPhone|iPod|BlackBerry|IEMobile|Opera Mini/i.test(navigator.userAgent);
    const isTablet = /iPad/.test(navigator.userAgent) || (navigator.userAgent.includes("Mac") && "ontouchend" in document);

    let gsapAnimation; // Variable to store GSAP animation

    // Clone items for smooth loop
    function cloneItems() {
      const fragment = document.createDocumentFragment();
      items.forEach(item => fragment.appendChild(item.cloneNode(true))); // Clone each item
      marqueeTrack.appendChild(fragment); // Append clones
    }

    // Preload images before starting animation
    function preloadImages() {
      return Promise.all(Array.from(marqueeTrack.querySelectorAll('img')).map(img => {
        return img.complete ? Promise.resolve() : new Promise(resolve => { img.onload = img.onerror = resolve; });
      }));
    }

    // GSAP animation for desktop
    function createDesktopAnimation() {
      if (gsapAnimation) gsapAnimation.kill(); // Kill existing animation if any

      const totalWidth = items.reduce((acc, item) => acc + item.offsetWidth, 0); // Total width of items
      gsap.set(marqueeTrack, { x: 0 }); // Set initial position
      gsapAnimation = gsap.to(marqueeTrack, {
        x: -totalWidth,
        duration: desktopDuration,
        ease: "none",
        repeat: -1, // Loop indefinitely
        modifiers: {
          x: gsap.utils.unitize(value => parseFloat(value) % totalWidth) // Wrap around at the end
        }
      });
    }

    // CSS animation for mobile/tablets
    function createMobileAnimation() {
      const totalWidth = items.reduce((acc, item) => acc + item.offsetWidth, 0); // Total width
      marqueeTrack.style.setProperty('--marquee-duration', `${mobileDuration}s`); // Animation duration
      marqueeTrack.style.setProperty('--marquee-width', `${totalWidth}px`); // Set total width
      marqueeTrack.classList.add('marquee-animate'); // Start CSS animation
    }

    // Handle window resize
    const handleResize = debounce(() => {
      if (isMobile || isTablet) {
        createMobileAnimation(); // Mobile animation
      } else {
        createDesktopAnimation(); // Desktop animation
      }
    }, 250);

    // Debounce function to limit resize calls
    function debounce(func, wait) {
      let timeout;
      return function (...args) {
        clearTimeout(timeout);
        timeout = setTimeout(() => func(...args), wait); // Run function after wait
      };
    }

    // Initialize marquee
    async function initMarquee() {
      cloneItems(); // Clone items
      await preloadImages(); // Wait for images to load
      if (isMobile || isTablet) {
        createMobileAnimation(); // Mobile/tablet animation
      } else {
        createDesktopAnimation(); // Desktop animation
      }
      window.addEventListener('resize', handleResize); // Resize event
    }

    // Intersection Observer for scroll-based activation
    const observer = new IntersectionObserver((entries) => {
      entries.forEach(entry => {
        if (isMobile || isTablet) {
          marqueeTrack.style.animationPlayState = entry.isIntersecting ? 'running' : 'paused'; // Pause/resume animation
        }
      });
    }, { threshold: 0.1 });

    observer.observe(marqueeTrack.parentElement); // Observe parent

    initMarquee(); // Start marquee
  });
</script>

<!-- Projects -->
<!-- Optional: Assign Load More to lists; skip if it's the only list on the page -->
<script>
  window.addEventListener("load", function () {  // Wait until all scripts/resources are loaded
    // Array of configurations for each project list
    const projectConfigs = [
      {
        listId: "#project-list-1",
        loadMoreId: "#project-load-1",
        paginationId: "#project-pagination-1",
      },
      {
        listId: "#project-list-2",
        loadMoreId: "#project-load-2",
        paginationId: "#project-pagination-2",
      }
    ];

    // Initialize FsLibrary for each project list
    projectConfigs.forEach(config => {
      const loadMoreProjects = new FsLibrary({
        collections: [
          {
            list: config.listId, // Target the project list by ID
            loadMore: config.loadMoreId, // Target the load more button by ID
            pagination: config.paginationId, // Target the pagination by ID
            template: ".project-list-item" // Assuming the template remains the same for all
          }
        ]
      });

      loadMoreProjects.init(); // Initialize FsLibrary for the current project list
    });
  }); 
</script>

<!-- Services -->
<!-- GSAP Accordion -->
<script>
  window.addEventListener("load", function () {  // Wait until all scripts/resources are loaded
    const accordions = document.querySelectorAll('.accordion--list-item'); // Select all accordion items
    let lastHoveredHeader = null; // Track the last hovered header
    let openAccordionHeader = null; // Track the currently open accordion header

    // Function to check if the device is mobile/tablet
    function isMobileOrTablet() {
      return window.innerWidth <= 991; // Breakpoint for tablets and below
    }

    // Loop through each accordion item
    accordions.forEach(accordion => {
      const header = accordion.querySelector('.accordion--header'); // Select the header
      const content = accordion.querySelector('.accordion--content'); // Select the content
      const divider = header.querySelector('.accordion--divider'); // Select the divider
      const accordionNr = header.querySelector('.accordion--nr'); // Select the accordion number
      const btnLine1 = header.querySelector('.accordion--btn-line-1'); // Select the first line of the button
      const btnLine2 = header.querySelector('.accordion--btn-line-2'); // Select the second line of the button

      // Set the initial state for content (collapsed)
      gsap.set(content, { height: '0px', opacity: 0, scale: 0.95, y: 10, overflow: 'hidden' });

      // Function to handle hover in and out animations for the line only
      const handleHoverLine = (hoverDivider, isIn) => {
        gsap.killTweensOf(hoverDivider); // Kill ongoing animations
        gsap.to(hoverDivider, { width: isIn ? '100%' : '0%', duration: 1.25, ease: 'power1.out' }); // Animate divider line
      };

      // Add hover animations if not on mobile/tablet
      if (!isMobileOrTablet()) {
        header.addEventListener('mouseenter', () => { // Hover on header
          if (header !== openAccordionHeader) {
            if (lastHoveredHeader && lastHoveredHeader !== header) {
              const lastHoverDivider = lastHoveredHeader.querySelector('.accordion--divider');
              if (lastHoveredHeader !== openAccordionHeader) {
                handleHoverLine(lastHoverDivider, false); // Handle last hovered header
              }
            }
            lastHoveredHeader = header; // Update last hovered header
            handleHoverLine(divider, true); // Handle current header
          }
        });

        header.addEventListener('mouseleave', () => { // Hover off header
          if (lastHoveredHeader === header && openAccordionHeader !== header) {
            handleHoverLine(divider, false); // Handle current header
            lastHoveredHeader = null; // Reset last hovered header
          }
        });
      }

      // Click event for the accordion header
      header.addEventListener('click', () => {
        const isOpen = parseFloat(gsap.getProperty(content, "height")) > 0; // Check if content is open

        // Disable scroll handling
        window.disableScrollHandling();

        // Close all other accordion contents
        accordions.forEach(item => {
          const itemContent = item.querySelector('.accordion--content');
          const itemHeader = item.querySelector('.accordion--header');
          const itemDivider = itemHeader.querySelector('.accordion--divider');
          const itemNr = itemHeader.querySelector('.accordion--nr');
          const itemBtnLine1 = itemHeader.querySelector('.accordion--btn-line-1');
          const itemBtnLine2 = itemHeader.querySelector('.accordion--btn-line-2');

          if (itemContent !== content) {
            gsap.to(itemContent, {
              height: '0px', opacity: 0, scale: 0.95, y: 10, duration: 0.5, ease: 'power1.inOut', onComplete: () => {
                gsap.set(itemContent, { height: '0px' }); // Ensure height is 0px after closing
              }
            }); // Close other contents
            gsap.to(itemDivider, { width: '0%', duration: 0.5, ease: 'power1.out' }); // Close other dividers
            gsap.to(itemNr, { color: "#c4c4c4", duration: 0.5, ease: 'power1.inOut' }); // Reset number color

            if (isMobileOrTablet()) {
              handleHoverLine(itemDivider, false); // Apply hover out on close
            }

            // Close button animation
            gsap.to(itemBtnLine1, { rotation: 0, duration: 0.75, ease: "expo.out" }); // Rotate line 1 to 0 degrees
            gsap.to(itemBtnLine1, { backgroundColor: "#13333a", duration: 0.75, delay: 0.25, ease: "ease.out" }); // Change BG color of line 1
            gsap.to(itemBtnLine2, { backgroundColor: "#13333a", duration: 0.75, delay: 0.25, ease: "ease.out" }); // Change BG color of line 2
          }
        });

        if (!isOpen) { // Open the clicked content if not already open
          gsap.to(content, {
            height: 'auto', opacity: 1, scale: 1, y: 0, duration: 0.5, ease: 'power1.out', onComplete: () => {
              gsap.set(content, { height: 'auto' }); // Ensure height is auto after opening
              if (isMobileOrTablet()) {
                header.scrollIntoView({ behavior: 'smooth', block: 'start' });
                setTimeout(() => {
                  window.scrollBy(0, -32); // Add padding (32px)
                }, 300); // Delay to ensure scrollIntoView completes first
              }
              // Enable scroll handling after animation
              setTimeout(window.enableScrollHandling, 1000); // Re-enable scroll handling after 1 second

              // Hide navigation when accordion is opened
              if (isMobileOrTablet()) {
                gsap.to(navHeader, {
                  y: -navHeader.offsetHeight,
                  duration: 0.8,
                  ease: 'power2.out'
                });
              }
            }
          });
          gsap.to(divider, { width: '100%', duration: 0.5, ease: 'power1.out' }); // Animate divider
          gsap.to(accordionNr, { color: "#2f7f90", duration: 0.5, ease: 'power1.inOut' }); // Change number color on open

          if (isMobileOrTablet()) {
            handleHoverLine(divider, true); // Apply hover on open
          }

          openAccordionHeader = header; // Set the opened accordion header

          // Open button animation
          gsap.to(btnLine1, { rotation: -90, duration: 0.75, ease: "bounce.out" }); // Rotate line 1 to -90 degrees
          gsap.to(btnLine1, { backgroundColor: "#2f7f90", duration: 0.5, delay: 0.25, ease: "bounce.out" }); // Change BG color of line 1
          gsap.to(btnLine2, { backgroundColor: "#2f7f90", duration: 0.5, delay: 0.25, ease: "bounce.out" }); // Change BG color of line 2
        } else { // Close the clicked content if already open
          gsap.to(content, {
            height: '0px', opacity: 0, scale: 0.95, y: 10, duration: 0.5, ease: 'power1.inOut', onComplete: () => {
              gsap.set(content, { height: '0px' }); // Ensure height is 0px after closing
              gsap.to(divider, { width: '0%', duration: 0.5, ease: 'power1.out' }); // Close divider
              gsap.to(accordionNr, { color: "#c4c4c4", duration: 0.5, ease: 'power1.inOut' }); // Reset number color on close

              if (isMobileOrTablet()) {
                handleHoverLine(divider, false); // Apply hover out on close
              }
              // Enable scroll handling after animation
              setTimeout(window.enableScrollHandling, 1000); // Re-enable scroll handling after 1 second
            }
          });

          openAccordionHeader = null; // Reset the opened accordion header

          // Close button animation
          gsap.to(btnLine1, { rotation: 0, duration: 0.75, ease: "expo.out" }); // Rotate line 1 to 0 degrees
          gsap.to(btnLine1, { backgroundColor: "#13333a", duration: 0.75, delay: 0.25, ease: "ease.out" }); // Change BG color of line 1
          gsap.to(btnLine2, { backgroundColor: "#13333a", duration: 0.75, delay: 0.25, ease: "ease.out" }); // Change BG color of line 2
        }
      });
    });
  });
</script>

In [ ]:
Temporary
_________

<!-- Process -->
<!-- GSAP Parallax -->
<script>
  window.addEventListener("load", function () {  // Wait until all scripts/resources are loaded
    gsap.registerPlugin(ScrollTrigger);

    // Check if the screen width is 992px or above
    function isDesktop() {
      return window.innerWidth >= 992;
    }

    // Function to animate each process item with unique animations
    function animateProcessItem(item, index) {
      const heading = item.querySelector('.process--heading');  // Get heading element
      const description = item.querySelector('.process--description');  // Get description element
      const smallIllustration = item.querySelector(`.illustration.cc--process-${index}--small`);  // Get small illustration
      const bigIllustration = item.querySelector(`.illustration.cc--process-${index}--big`);  // Get big illustration

      // Animate heading
      if (heading) {
        gsap.fromTo(heading,
          { y: 100 },  // Start position
          {
            y: 0,  // End position
            scrollTrigger: {
              trigger: item,
              start: "top 80%",  // Start animation
              end: "bottom 20%",  // End animation
              scrub: true  // Smooth scrubbing
            }
          });
      }

      // Animate description
      if (description) {
        gsap.fromTo(description,
          { y: 150 },  // Start position
          {
            y: 0,  // End position
            scrollTrigger: {
              trigger: item,
              start: "top 80%",  // Start animation
              end: "bottom 20%",  // End animation
              scrub: true  // Smooth scrubbing
            }
          });
      }

      // Animate small illustration with unique settings
      if (smallIllustration) {
        let smallAnim = {};
        switch (index) {
          case 1:
            smallAnim = { y: 300, rotation: -60, scale: 0.6 };  // Process 1 small
            break;
          case 2:
            smallAnim = { y: 250, rotation: -50, scale: 0.7 };  // Process 2 small
            break;
          case 3:
            smallAnim = { y: 200, rotation: -40, scale: 0.8 };  // Process 3 small
            break;
          case 4:
            smallAnim = { y: 150, rotation: -30, scale: 0.9 };  // Process 4 small
            break;
          case 5:
            smallAnim = { y: 100, rotation: -20, scale: 1.0 };  // Process 5 small
            break;
          case 6:
            smallAnim = { y: 50, rotation: -10, scale: 1.1 };  // Process 6 small
            break;
        }
        gsap.fromTo(smallIllustration,
          smallAnim,  // Start position
          {
            y: 0, rotation: 0, scale: 1,  // End position
            scrollTrigger: {
              trigger: item,
              start: "top 80%",  // Start animation
              end: "bottom 20%",  // End animation
              scrub: true  // Smooth scrubbing
            }
          });
      }

      // Animate big illustration with unique settings
      if (bigIllustration) {
        let bigAnim = {};
        switch (index) {
          case 1:
            bigAnim = { y: 300, rotation: 60, scale: 0.6 };  // Process 1 big
            break;
          case 2:
            bigAnim = { y: 250, rotation: 50, scale: 0.7 };  // Process 2 big
            break;
          case 3:
            bigAnim = { y: 200, rotation: 40, scale: 0.8 };  // Process 3 big
            break;
          case 4:
            bigAnim = { y: 150, rotation: 30, scale: 0.9 };  // Process 4 big
            break;
          case 5:
            bigAnim = { y: 100, rotation: 20, scale: 1.0 };  // Process 5 big
            break;
          case 6:
            bigAnim = { y: 50, rotation: 10, scale: 1.1 };  // Process 6 big
            break;
        }
        gsap.fromTo(bigIllustration,
          bigAnim,  // Start position
          {
            y: 0, rotation: 0, scale: 1,  // End position
            scrollTrigger: {
              trigger: item,
              start: "top 80%",  // Start animation
              end: "bottom 20%",  // End animation
              scrub: true  // Smooth scrubbing
            }
          });
      }
    }

    // Select all process items
    const processItems = document.querySelectorAll('.process--list .w-dyn-item');

    // Ensure initial visibility and apply animation to each process item if screen width is 992px or above
    if (isDesktop()) {
      processItems.forEach((item, index) => {
        item.style.visibility = 'visible';  // Ensure visibility
        animateProcessItem(item, index + 1);  // Unique class names
      });
    }

    // Add a resize event listener to apply animations dynamically
    window.addEventListener('resize', () => {
      if (isDesktop()) {
        processItems.forEach((item, index) => {
          item.style.visibility = 'visible';  // Ensure visibility
          animateProcessItem(item, index + 1);  // Unique class names
        });
      } else {
        processItems.forEach((item) => {
          gsap.killTweensOf(item.querySelectorAll('.process--heading, .process--description, .illustration'));
          item.style.visibility = '';  // Reset visibility
        });
      }
    });
  });
</script>

In [ ]:
CSS | Responsive
________________

<style>
  /* Hero */
  /* Illustration size */
  /* Tablet */
  @media only screen and (max-width: 991px) and (min-width: 720px) {
    .illustration.home--hero-cursor {
      /* Size */
      width: 4.25rem;
      height: 4.25rem;

      /* Position */
      position: absolute;
      top: -1.3rem;
      right: auto;
      bottom: auto;
      left: 6rem;
    }

    .illustration.home--hero-male {
      /* Size */
      width: 9rem;
      height: 9rem;

      /* Position */
      position: absolute;
      top: -4.75rem;
      right: auto;
      bottom: auto;
      left: 9.25rem;
    }

    .illustration.home--hero-female {
      /* Size */
      width: 9.5rem;
      height: 9.5rem;

      /* Position */
      position: absolute;
      top: auto;
      right: -7.25rem;
      bottom: 0.25rem;
      left: auto;
    }
  }

  /* Mobile, Large */
  @media only screen and (max-width: 479px) and (min-width: 410px) {
    .illustration.home--hero-cursor {
      /* Size */
      width: 3.25rem;
      height: 3.25rem;

      /* Position */
      position: absolute;
      top: -0.8rem;
      right: auto;
      bottom: auto;
      left: 6.5rem;
    }

    .illustration.home--hero-male {
      /* Size */
      width: 6.5rem;
      height: 6.5rem;

      /* Position */
      position: absolute;
      top: -4.5rem;
      right: auto;
      bottom: auto;
      left: 8.5rem;
    }

    .illustration.home--hero-female {
      /* Size */
      width: 6.5rem;
      height: 6.5rem;

      /* Position */
      position: absolute;
      top: auto;
      right: -5rem;
      bottom: -1rem;
      left: auto;
    }
  }
</style>


<style>
  /* Projects */
  /* Heading size */
  /* Tablet */
  @media (min-width: 660px) {
    .heading--w.mobile {
      display: block;
    }

    .heading--w.mobile--small {
      display: none;
    }
  }

  /* Mobile */
  @media (max-width: 659px) {
    .heading--w.mobile {
      display: none;
    }

    .heading--w.mobile--small {
      display: block;
    }
  }

  /* Base CSS for all CMS list items */
  /* Desktop */
  .w-dyn-item[cms-item^="project-"] .project--image {
    background-size: auto 100%;
    /* Custom size: Auto width, 100% height */
    background-position: 50% 50%;
    /* Center position */
    background-repeat: no-repeat;
    /* No tile */
  }

  /* Tablet */
  @media (min-width: 480px) and (max-width: 991px) {
    .w-dyn-item[cms-item^="project-"] .project--image {
      background-size: 100% auto;
      /* Custom size: Auto width, 100% height */
      background-position: 50% 50%;
      /* Center position */
      background-repeat: no-repeat;
      /* No tile */
    }
  }

  /* Mobile */
  @media (max-width: 479px) {
    .w-dyn-item[cms-item^="project-"] .project--image {
      background-size: auto 100%;
      /* Custom size: Auto width, 100% height */
      background-position: 50% 50%;
      /* Center position */
      background-repeat: no-repeat;
      /* No tile */
    }
  }

  /* Overrides for individual CMS list items */
  /* Desktop */
  .w-dyn-item[cms-item="project-1"] .project--image {
    background-size: auto 150%;
    background-position: 35% 35%;
  }

  .w-dyn-item[cms-item="project-4"] .project--image {
    background-size: auto 135%;
    background-position: 50% 55%;
  }

  .w-dyn-item[cms-item="project-5"] .project--image {
    background-size: auto 130%;
    background-position: 50% 0%;
  }

  /* Tablet */
  @media (min-width: 480px) and (max-width: 991px) {
    .w-dyn-item[cms-item="project-1"] .project--image {
      background-size: 135% auto;
      background-position: 55% 40%;
    }

    .w-dyn-item[cms-item="project-4"] .project--image {
      background-size: auto 135%;
      background-position: 50% 75%;
    }

    .w-dyn-item[cms-item="project-7"] .project--image {
      background-size: auto 110%;
      background-position: 70% 100%;
    }
  }

  /* Mobile */
  @media (max-width: 479px) {
    .w-dyn-item[cms-item="project-1"] .project--image {
      background-size: auto 130%;
      background-position: 47.5% 37.5%;
    }

    .w-dyn-item[cms-item="project-7"] .project--image {
      background-size: auto 110%;
      background-position: 70% 100%;
    }
  }
</style>

In [ ]:
CSS | Styling
_____________

<style>
  /* Hero */
  /* B2B Caption - Dashed border */
  .caption--w.cc--dashed-border {
    position: relative;
    /* Position relative for pseudo-elements */
    display: inline-block;
    /* Fit content */
    box-sizing: border-box;
    /* Include padding and border in dimensions */
    border-radius: 1rem;
    /* Rounded corners */
    overflow: hidden;
    /* Contain the pseudo-element */
  }

  .caption--w.cc--dashed-border::before {
    content: '';
    /* Empty element for border */
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    border: 0.1875rem solid transparent;
    border-radius: inherit;
    pointer-events: none;
    box-sizing: border-box;
    background:
      /* Dashed border effect */
      linear-gradient(90deg, #13333A 1rem, transparent 1rem) repeat-x,
      linear-gradient(180deg, #13333A 1rem, transparent 1rem) repeat-y,
      linear-gradient(90deg, transparent 1rem, #13333A 1rem) repeat-x,
      linear-gradient(180deg, transparent 1rem, #13333A 1rem) repeat-y;
    background-size: calc(2rem + 0.1875rem) 0.1875rem,
      /* Size for horizontal dash */
      0.1875rem calc(2rem + 0.1875rem),
      /* Size for vertical dash */
      calc(2rem + 0.1875rem) 0.1875rem,
      /* Size for bottom-right horizontal dash */
      0.1875rem calc(2rem + 0.1875rem);
    /* Size for bottom-right vertical dash */
    background-position: top left, top left, bottom right, bottom right;
    /* Position dashes */
  }

  /* GSAP Marquee */
  /* CSS Animation for Mobile */
  @keyframes marquee {
    0% {
      transform: translateX(0);
      /* Start position */
    }

    100% {
      transform: translateX(calc(-1 * var(--marquee-width)));
      /* End position */
    }
  }

  .marquee--track {
    display: flex;
    /* Flex container */
    will-change: transform;
    /* Optimize for performance */
  }

  .marquee-animate {
    animation: marquee var(--marquee-duration) linear infinite;
    /* Animation settings */
    -webkit-animation: marquee var(--marquee-duration) linear infinite;
    /* Vendor prefix for WebKit */
    -moz-animation: marquee var(--marquee-duration) linear infinite;
    /* Vendor prefix for Mozilla */
    -o-animation: marquee var(--marquee-duration) linear infinite;
    /* Vendor prefix for Opera */
  }
</style>


<style>
  /* Projects */
  /* CMS list - Alternate direction per item */
  @media (min-width: 992px) {

    /* Even items: reverse the direction */
    .project--list .w-dyn-item:nth-child(even) .project--card {
      display: flex;
      flex-direction: row-reverse;
      /* Reverse the order */
    }

    /* Ensure the internal content remains left-to-right */
    .project--list .w-dyn-item:nth-child(even) .project--image,
    .project--list .w-dyn-item:nth-child(even) .project--about {
      direction: ltr;
      /* Maintain normal text direction */
    }
  }
</style>


<style>
  /* Process */
  /* Base CSS for all CMS list items */
  .process--list-item {
    display: flex;
    flex-direction: column;
    width: 100%;
    margin-bottom: 2rem;
    /* Adjust spacing between items as needed */
  }

  /* Overrides for individual CMS list items */
  /* Desktop */
  @media (min-width: 992px) {

    /* Align items based on their data-item attribute */
    /* Align left */
    .process--list-item[cms-item="process-1"],
    .process--list-item[cms-item="process-4"],
    {
    align-items: flex-start;
  }

  /* Align center */
  .process--list-item[cms-item="process-3"],
  .process--list-item[cms-item="process-6"] {
    align-items: center;
  }

  /* Align right */
  .process--list-item[cms-item="process-2"],
  .process--list-item[cms-item="process-5"] {
    align-items: flex-end;
  }
  }
</style>